# Interactive proofs

In this chapter we explain what interactive proofs are.

Virtually every part of the workshop depends on this concept, so this chapter is a must read.

Read the TL;DR part if you are short on time. Read the rest for more info.

# TL;DR

An interactive proof is a game between two players. We call them Peggy and Victor.

Peggy wants to convince Victor of a statement.

Victor isn't gullible so he wants to see evindence before accepting the statement as true.

Peggy wins if she can convince Victor. Victor wins if he accepts true statements and rejects false statements.

There is the good case where Peggy is honest. Her statement is true. Victor wins if he accepts, which means Peggy also wins. Everybody is happy 😊

And then there is the evil case where Peggy is lying. Her statement is false. The winning conditions are asymmetric, so only one party can win this time. If Victor exposes the lie, he wins and Peggy loses. If Peggy fools Victor into accepting, she wins and he loses. This feels like a zero-sum game 😕

Interactive proofs are designed to make the evil case expensive for Peggy. She is motivated to be honest and everybody wins.

# Jupyter setup

Run the following snippet to set up your jupyter notebook for the workshop.

In [ ]:
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Statements

A statement is any precise mathematical fact that can be true or false.

"1 + 1 = 2"

"2 + 2 = 5"

_Statements don't have to be true._

"A cube is a 3D object"

"This puzzle has a solution"

"I know the solution to this puzzle"

The last one is interesting because it talks about knowledge of something. Later we will make this mathematically precise.

In [ ]:
# We will use this statement for our exercises

# "I know a number x such that 2 + x = 5"
def statement(witness):
    return 2 + witness == 5

# Does an honest Prover always win?

If the Prover is honest, then it should be able to convince the Verifier. The Verifier has to play along instead of rejecting every attempt.

This should work almost all the time. Prover and Verifier are probabilistic algorithms.

We call this **completeness** and it is one of three important properties of interactive proof systems.

## Exercise

What is a simple protocol for our statement that is complete?

In [ ]:
class CompleteProver:
    def send(self):
        pass  # TODO: What to send?
        
class CompleteVerifier:
    def verify(self, x):
        pass  # TODO: How to verify?
        
prover = CompleteProver()
verifier = CompleteVerifier()
x = prover.send()

if verifier.verify(x):
    print("Convinced 👌")
else:
    print("Not convinced... 🤨")

# Does a vigilant Verifier never lose?

If the Prover is dishonest, it shouldn't be able to deceive the Verifier. Even if the Prover disregards the protocol. The Verifier has to do minimal checking instead of accepting every attempt.

Again, this should work most of the time.

We call this **soundness** and it is the second important property of interactive proof systems.

## Exercise

What is a simple protocol for our statement that is sound but incomplete?

In [ ]:
class SoundProver:
    def send(self):
        pass  # TODO: What to send?
        
class SoundVerifier:
    def verify(self, x):
        pass  # TODO: How to verify?
        
prover = SoundProver()
verifier = SoundVerifier()
x = prover.send()

if verifier.verify(x):
    print("Convinced 👌")
else:
    print("Not convinced... 🤨")

## Exercise

What is a simple protocol for our statement that is sound and complete?

Combining soundness and completeness makes it much harder to "cheat" in protocol design.

In [ ]:
class CorrectProver:
    def send(self):
        pass  # TODO: What to send?
        
class CorrectVerifier:
    def verify(self, x):
        pass  # TODO: How to verify?
        
prover = CorrectProver()
verifier = CorrectVerifier()
x = prover.send()

if verifier.verify(x):
    print("Convinced 👌")
else:
    print("Not convinced... 🤨")

# Knowledge of something

Prover and Verifier are algorithms. What does it mean for them to "know" something?

Think of it this way: **An algorithm knows what it can (easily) compute.**

Obviously it knows its initial data and anything passed to it from the outside world.

It learns new knowledge by computing new values based on what it already knows. This makes sense: Humans also make these deriviations.

Is there a commonly accepted definition for an "easy" computation? Yes! **Polynomial computations**.

Polynomial computations _(the complexity class P)_ are seen as easy / feasible / efficient. Addition is easy. Finding a path between two points in a graph is easy. Verifying the solution to a sudoku is easy.

The opposite are exponential computations _(the complexity class NP)_ which are seen as hard / infeasible / inefficient. Finding the shortest path that visits all points in a graph exactly once is hard. Coming up with a solution to a sudoku is hard. Computing the discrete logarithm of a curve point is hard.

An algorithm can learn polynomially, but not exponentially. Exponential learning would be cheating. A human equivalent would be to come up with mathematical proofs of artistic masterpieces on the spot. Some humans do that, sometimes, but we don't want to assume that our algorithms are geniuses! Also, exponential power would far surpass even the greatest human geniuses.

# Extracting knowledge as proof of knowledge

The Verifier wants to accept only if the Prover really knows what it claims to know. How can we make sure of that?

We do a thought experiment: The Prover is a black box that claims to contain a secret. We are the Verifier and would like to learn the secret. If we can interact with this box through the protocol to learn the secret, then the box must have contained it to begin with!

In the physical world this is obvious, but there is some interesting logic in the mathematical world.

In the beginning, the Verifier doesn't know the secret. No polynomial computation leads to it. Then the Verifier interacts with the Prover in a _polynomial_ way. The Verifier is not allowed to do any genius-level exponential stuff. This interaction is one of the aforementioned polynomial computations, but this time the Prover is there. The Verifier learns the secret. Ergo, the secret came from the Prover!

# Zero knowledge

The Prover wants to convince the Verifier without revealing any information. This seems impossible.

Let's rephrase the task: The Verifier is not supposed to learn anything that it doesn't already know.

We do another thought experiment: At the start of the protocol, the Verifier knows what it can compute in polynomial time, without access to the Prover. We show that anything the Verifier might learn during the protocol, the Verifier already knew before the protocol started!

Our protocols are probabilistic, so our situation is slightly more complicated.

Prover and Verifier exchange messages, called the transcript. This randomly changes every time, so we have a random distribution of transcripts. The Verifier uses some algorithm to extract information from this distribution.

If the incoming distribution of transcripts is the same, then the outgoing distribution of extracted information is the same.

To show that the Verifier already knew everything it could extract, we show that the Verifier can compute identical-looking transcripts. This happens without access to the Prover and in polynomial time.

Ergo, the Verifier ends up extracting information that it already knows and our protocol is zero-knowledge.